Testing te decoder training

In [26]:
import torch
from models import custom_models

import torch.nn as nn
import torch.optim as optim
import torchvision

import copy
import numpy as np
import scipy.stats as ss
import scipy
import h5py
import random
import argparse
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pylab as plt

In [91]:
import socket
if socket.gethostname()[0:4] in  ['node','holm','wats']:
    path_prefix = '/rigel/issa/users/Tahereh/Research'
elif socket.gethostname() == 'SYNPAI':
    path_prefix = '/hdd6gig/Documents/Research'
elif socket.gethostname()[0:2] == 'ax':
    path_prefix = '/scratch/issa/users/tt2684/Research'
    plt.switch_backend('agg')
elif socket.gethostname() == 'turing':
    path_prefix = '/home/tahereh/Documents/Research'


imagesetdir = path_prefix+'/Data/'
resultsdir = path_prefix+'/Results/Toy_models/'

class Args:
    dum=None

args = Args()
args.batch_size=256
args.no_cuda = False
args.seed = 0
args.n_classes = 10
args.dataset = 'MNIST'
args.arche = 'FullyConnectedF'
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

use_cuda = True
# data loader
kwargs = {'num_workers': 0, 'pin_memory': True, 'drop_last':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=True, download=True,
                    transform=transforms.Compose([
                        transforms.Resize(32),
                        transforms.ToTensor(),
                       
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=False, transform=transforms.Compose([
                        transforms.Resize(32),
                        transforms.ToTensor(),
                        
                    ])),
    batch_size=args.batch_size, shuffle=False, **kwargs)

In [92]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [93]:
model = custom_models.FullyConnectedF(hidden_layers=[256,256,10], nonlinearfunc='relu', input_length=32*32, algorithm='FA').cuda()

In [94]:
state_dict = torch.load('/home/tahereh/Documents/Research/Results/Symbio/Symbio/Oct02-21-26_MNIST_e6507d0d57_RMSpropRMSpropMNISTFullyConnE150_805/checkpointe_BP.pth.tar')['state_dict']
# state_dict = torch.load('/home/tahereh/Documents/Research/Results/Symbio/Symbio/Oct03-07-02_MNIST_30fb700eaa_None_519/checkpointe_BP.pth.tar')['state_dict']


state_dict_model = {}
for k in state_dict.keys():
    state_dict_model.update({k.replace('module.',''):state_dict[k]})

In [95]:
state_dict.keys()

odict_keys([&#39;module.fc_0.weight&#39;, &#39;module.fc_0.weight_feedback&#39;, &#39;module.fc_1.weight&#39;, &#39;module.fc_1.weight_feedback&#39;, &#39;module.fc_2.weight&#39;, &#39;module.fc_2.weight_feedback&#39;])

In [96]:
model.load_state_dict(state_dict_model)

&lt;All keys matched successfully&gt;

In [105]:

model.eval()
# with torch.no_grad():
for i, (images, target) in enumerate(test_loader):


    images = images.cuda()
    target = target.cuda()
    
    
    images = images.expand(-1, 1, -1, -1) #images.expand(-1, 3, -1, -1)
    
    # ----- encoder ---------------------
                
    # compute output
    latents, o = model(images)
    acc1, acc5 = accuracy(o, target, topk=(1, 5))
    print('orig',acc1[0].item())

    n_latents = latents.view(latents.shape[0], -1).shape[-1]
    decoder = nn.Linear(n_latents, args.n_classes).cuda()
    optimizerD = torch.optim.SGD(decoder.parameters(), lr=0.1, weight_decay=1e-3)
    criterionD = nn.CrossEntropyLoss()

    for ep in range(5):
        running_lossD = 0
        for iD, (imagesD, targetD) in enumerate(train_loader):
        
            imagesD = imagesD.cuda()
            targetD = targetD.cuda()   

        
            if ('MNIST' in args.dataset) and args.arche[0:2]!='FC':
                imagesD= imagesD.expand(-1, 1, -1, -1) #images= images.expand(-1, 3, -1, -1) 

            latentsD, _ = model(imagesD)

            optimizerD.zero_grad()
            outputsD = decoder(latentsD.view(latentsD.shape[0], -1).detach())
            lossD = criterionD(outputsD, targetD)
            lossD.backward()
            optimizerD.step()

            running_lossD += lossD.item()

        # print(running_lossD/(iD+1))

        output = decoder(latents.view(latents.shape[0], -1).detach())

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        print('decoder', acc1[0].item())


orig 98.828125
decoder 98.828125
decoder 98.828125
decoder 98.828125
decoder 98.828125
decoder 99.21875
orig 97.65625
decoder 98.046875
decoder 97.65625
decoder 97.65625
decoder 97.65625
decoder 97.65625
orig 98.828125
decoder 98.4375
decoder 98.4375
decoder 98.4375
decoder 98.4375
decoder 98.4375
orig 98.4375
decoder 98.046875
decoder 98.046875
decoder 98.046875


KeyboardInterrupt: 

In [76]:
args


&lt;__main__.Args at 0x7fe46523f490&gt;

## Testing BP in modules

In [54]:
import torch
from torch import autograd
import torch.nn as nn
from modules import BiHebb_modules as customized_modules

In [55]:
Linear = customized_modules.Linear
net = Linear(64, 16, False, 'FA')
inputs = torch.randn(256, 64)
targets = torch.randn(256, 16)
outputs = net(inputs)

In [56]:
net.weight

Parameter containing:
tensor([[ 0.0685,  0.1118, -0.0390,  ..., -0.1008, -0.0246, -0.0348],
        [ 0.0572, -0.0658,  0.0760,  ...,  0.1161,  0.0440, -0.0455],
        [ 0.0815,  0.0741,  0.0345,  ...,  0.0596, -0.0076, -0.0036],
        ...,
        [ 0.0339, -0.0810, -0.0373,  ...,  0.1171,  0.0035,  0.0786],
        [-0.0708, -0.0919, -0.0110,  ..., -0.0797, -0.1178, -0.0891],
        [ 0.0156,  0.0254,  0.0426,  ...,  0.0513, -0.0196,  0.1123]],
       requires_grad=True)

In [57]:
net.weight_feedback

Parameter containing:
tensor([[ 0.0572, -0.0218,  0.0713,  ..., -0.0719, -0.0940, -0.1124],
        [-0.1021,  0.0163, -0.0337,  ...,  0.0570,  0.0086,  0.0826],
        [-0.0693, -0.0021,  0.0284,  ...,  0.0683,  0.0405,  0.1027],
        ...,
        [-0.0033, -0.0145, -0.0922,  ..., -0.0150, -0.0409, -0.0853],
        [-0.0288,  0.0877,  0.1063,  ..., -0.0343, -0.0526, -0.0030],
        [ 0.0782,  0.1013, -0.0678,  ...,  0.1000, -0.1146,  0.0892]])

In [58]:
Conv2d = customized_modules.Conv2d
net = Conv2d(64, 16, 3, algorithm='FA')
inputs = torch.randn(256, 64, 32, 32)
outputs = net(inputs)
net.weight[0,0]

tensor([[-0.0041, -0.0059, -0.0162],
        [-0.0341,  0.0090,  0.0318],
        [ 0.0396, -0.0045, -0.0339]], grad_fn=&lt;SelectBackward&gt;)

In [59]:
net.weight_feedback[0,0]

tensor([[-0.0350,  0.0119, -0.0236],
        [ 0.0006, -0.0341, -0.0392],
        [-0.0068,  0.0342,  0.0386]])

## Developing Target Prop

In [60]:
import torch
from torch import autograd
import torch.nn as nn
from modules import TP_modules as customized_modules

In [61]:
Linear = customized_modules.Linear
net = Linear(64, 16, False, 'TP')
inputs = torch.randn(256, 64)
targets = torch.randn(256, 16)
outputs = net(inputs)
print(outputs.shape)
loss = nn.MSELoss()(outputs , targets)
loss.backward()

# autograd.grad(loss, net.weight)[0].shape

torch.Size([256, 16])


In [62]:

import torch.nn as nn
import torch.optim as optim
import torchvision

import copy
import numpy as np
import scipy.stats as ss
import scipy
import h5py
import random
import argparse
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pylab as plt

In [ ]:
import socket
if socket.gethostname()[0:4] in  ['node','holm','wats']:
    path_prefix = '/rigel/issa/users/Tahereh/Research'
elif socket.gethostname() == 'SYNPAI':
    path_prefix = '/hdd6gig/Documents/Research'
elif socket.gethostname()[0:2] == 'ax':
    path_prefix = '/scratch/issa/users/tt2684/Research'
    plt.switch_backend('agg')
elif socket.gethostname() == 'turing':
    path_prefix = '/home/tahereh/Documents/Research'


imagesetdir = path_prefix+'/Data/'
resultsdir = path_prefix+'/Results/Toy_models/'

class Args:
    dum=None

args = Args()
args.batch_size=256
args.algorithm = 'TP' #'TP' #'BP'
args.no_cuda = False
args.seed = 0

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

use_cuda = True
# data loader
kwargs = {'num_workers': 0, 'pin_memory': True, 'drop_last':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=True, download=True,
                    transform=transforms.Compose([
                        transforms.Resize(32),
                        transforms.ToTensor(),
                       
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=False, transform=transforms.Compose([

                        transforms.Resize(32),
                        transforms.ToTensor(),
                        
                    ])),
    batch_size=args.batch_size, shuffle=False, **kwargs)

# transforms.Normalize((0.1307,), (0.3081,))
n_layers = 3
algorithm = args.algorithm #'inverse'#'forward'#'inverse'#'BP'
n_dataloader = 100000
batch_size = args.batch_size


class ReLUGrad(nn.Module):
    def __init__(self):
        super(ReLUGrad, self).__init__()
    def forward(self, grad_output, input):
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

class Forward(nn.Module):
    def __init__(self):
        super(Forward, self).__init__()
        self.fc_0 = Linear(1024, 256, bias=False, algorithm=algorithm)
        self.fc_1 = Linear(256, 10, bias=False, algorithm=algorithm)
        self.relu = nn.ReLU()

    def forward(self, x):
        xrelu = self.fc_0(x)
        x0 = self.relu(xrelu)
        x1 = self.fc_1(x0)

        return x1 #, [x, x0, x1], xrelu

class Backward(nn.Module):
    def __init__(self):
        super(Backward, self).__init__()
        self.fc_1 = Linear(10, 256, bias=False, algorithm=algorithm)
        self.fc_0 = Linear(256, 1024, bias=False, algorithm=algorithm)
        self.grelu = ReLUGrad()

    def forward(self, x, x0):
        x1 = self.fc_1(x)
        x1 = self.grelu(x1, x0)
        x0 = self.fc_0(x1)

        return x0#, [x1,  x]

def transpose_weights(state_dict):

    state_dict_new = {}
    for k, item in state_dict.items():
        state_dict_new.update({k: item.t()})
    return state_dict_new



modelF = Forward().cuda() # main model
modelB = Backward().cuda() # backward network to compute gradients for modelF

modelC = Forward().cuda() # Forward Control model to compare to BP
modelC.load_state_dict(modelF.state_dict())
# start symmetric
# modelB.load_state_dict(transpose_weights(modelF.state_dict()) )
modelE = Forward().cuda()

optimizerC = optim.RMSprop(modelC.parameters(), lr=1e-3, weight_decay=1e-6)
optimizerF = optim.RMSprop([p for n,p in modelF.named_parameters() if 'feedback' not in n],  lr=1e-3, weight_decay=1e-6)
optimizerB = optim.RMSprop([p for n,p in modelB.named_parameters() if 'feedback' not in n],  lr=1e-3, weight_decay=1e-6)
optimizerB_TP = optim.RMSprop([p for n,p in modelF.named_parameters() if 'feedback' in n],  lr=1e-3, weight_decay=1e-6)


criterionF = nn.CrossEntropyLoss() #
# criterionB = nn.MSELoss() #

n_classes = 10
onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()


In [70]:
for epoch in range(10):
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.cuda()
        targets = targets.cuda()

        onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()
        onehot.zero_()
        onehot.scatter_(1, targets.view(train_loader.batch_size,-1), 1)
        onehot.requires_grad = True

        images = images.view(images.shape[0], -1)
        out = modelF(images)
        loss = criterionF(out, targets)
        optimizerB_TP.zero_grad()
        loss.backward()
        optimizerB_TP.step()

        
        optimizerF.zero_grad()
        onehot.backward(torch.ones_like(onehot))
        optimizerF.step()

        running_loss += loss.item()
    print(epoch, running_loss/len(train_loader))


0 0.5676737737197143
1 0.567823022349268
2 0.5681442454075202
3 0.5692038304275937
4 0.5727376921309365


KeyboardInterrupt: 

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.cuda()
        targets = targets.cuda()

    

        images = images.view(images.shape[0], -1)
        out = modelF(images)
        loss = criterionF(out, targets)

        
        optimizerF.zero_grad()
        loss.backward()
        optimizerF.step()

        running_loss += loss.item()
    print(epoch, running_loss/len(train_loader))
